## Prerequisite Packages

In [2]:
# Based on https://www.tensorflow.org/datasets/keras_example
# use pip install <package> to find any packages that are missing
import keras         # for fitting DNNs
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt

from random import randint
from numpy import array
from numpy import argmax
from numpy import array_equal
from keras.models import Sequential
from keras.layers import LSTM,Dense,TimeDistributed,RepeatVector
from keras.utils import plot_model
import pydotplus,pydot
from keras.utils.vis_utils import model_to_dot
keras.utils.vis_utils.pydot = pydot


## Extracting data

In [3]:
# data from: www.manythings.org/anki/
# define training data
data=pd.read_csv('/Users/Ellina/Desktop/DS552/Lecture Note/Week9-RNNs and NLP/deu-eng/deu.txt',sep='\t',header=None).iloc[:,[0,1]]
data.columns=['english','italian']
# shuffling
data = data.sample(frac = 1) 
train_pct=0.8
# words begin with "\t" and end with "\n"
data['english']=[['\t']+line.replace('.','').replace('!','').replace('?','').split(' ')+['\n'] for line in data['english'].values]
data['italian']=[['\t']+line.replace('.','').replace('!','').replace('?','').split(' ')+['\n'] for line in data['italian'].values]


## Training and testing data

In [9]:

# train data
train_data=data.iloc[:int(train_pct*len(data)),]
# train model
embedding_eng = Word2Vec(train_data['english'].values, size=300, min_count=1)
embedding_it = Word2Vec(train_data['italian'].values, size=300, min_count=1)
max_sent_eng = 5#np.max([len(sentence) for sentence in train_data['english'].values])
max_sent_it = 5#np.max([len(sentence) for sentence in train_data['italian'].values])
# embedding using training data-based model
train_data['english_emb']=[[embedding_eng[word] for word in sentence]  if len(sentence)<=max_sent_eng else np.nan for sentence in train_data['english'].values]
train_data['italian_emb']=[[embedding_it[word]  for word in sentence]  if len(sentence)<=max_sent_it else np.nan for sentence in train_data['italian'].values]
train_data=train_data.dropna()
train_data['english_emb'] = [sentence+[embedding_eng['\n']]*(max_sent_eng-len(sentence)) for sentence in train_data['english_emb'].values]
train_data['italian_emb'] = [sentence+[embedding_it['\n']]*(max_sent_it-len(sentence)) for sentence in train_data['italian_emb'].values]


test_data=data.iloc[int(train_pct*len(data)):,].reset_index(drop=True)
test_data['english_emb']=[[embedding_eng[word] for word in sentence if word in embedding_eng.wv.vocab] for sentence in test_data['english'].values]
test_data['italian_emb']=[[embedding_it[word] for word in sentence if word in embedding_it.wv.vocab] for sentence in test_data['italian'].values]
test_data=test_data.loc[[len(line['english'])<=max_sent_eng and len(line['italian'])<=max_sent_it for n,line in test_data.iterrows()],]
# remove data if words not in embedding method
test_data=test_data.loc[[len(line['english'])==len(line['english_emb']) and len(line['italian'])==len(line['italian_emb']) for n,line in test_data.iterrows()],]
test_data['english_emb'] = [sentence+[embedding_eng['\n']]*(max_sent_eng-len(sentence)) for sentence in test_data['english_emb'].values]
test_data['italian_emb'] = [sentence+[embedding_it['\n']]*(max_sent_it-len(sentence)) for sentence in test_data['italian_emb'].values]


<ipython-input-9-02ccf56297c2>:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  train_data['english_emb']=[[embedding_eng[word] for word in sentence]  if len(sentence)<=max_sent_eng else np.nan for sentence in train_data['english'].values]
<ipython-input-9-02ccf56297c2>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['english_emb']=[[embedding_eng[word] for word in sentence]  if len(sentence)<=max_sent_eng else np.nan for sentence in train_data['english'].values]
<ipython-input-9-02ccf56297c2>:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  train_data['german_emb']=[[embedding_

## Plotting embedded training data

## Embedding demonstrates that translation is a transformation of word embeddings, but not a trivial mapping!

In [ ]:

eng_words = list(embedding_eng.wv.vocab)[:1000]#['Hi!', 'Run!', 'Who?', 'Jump.', 'Stay.', 'Go', 'ran.', 'see.', 'try.']#list(embedding_eng.wv.vocab)[:20]
it_words=list(embedding_it.wv.vocab)[:1000]#["Ciao!", "Corri!", "Chi?", "Salta.", "Resta.", "Vai", "corri", "vedi", "prova"] 
#print("Ciao!" in embedding_it.wv.vocab)
tsne =TSNE(n_components=2)
result = tsne.fit_transform(np.concatenate((embedding_it[it_words],embedding_eng[eng_words])))
result_eng = result[:len(eng_words)]#tsne.fit_transform()
result_it = result[len(eng_words):]
# create a scatter plot of the projection
fig, ax = plt.subplots(1, 1,figsize=(10,10))
plt.scatter(result_eng[:, 0], result_eng[:, 1],color='k',marker='.',alpha=0.2)
plt.scatter(result_it[:, 0], result_it[:, 1],color='r',marker='s',alpha=0.1)
plt.ylim([-70,70])
plt.xlim([-50,40])
for i, word in enumerate(it_words[:50]):
    plt.annotate(word, xy=(result_it[i, 0], result_it[i, 1]),color='darkred')
for i, word in enumerate(eng_words[:50]):
    plt.annotate(word, xy=(result_eng[i, 0], result_eng[i, 1]),color='k')
plt.show()
    

## Translation Model

### Training Model

In [1]:

# Based on: https://machinelearningmastery.com/encoder-decoder-attention-sequence-to-sequence-prediction-keras/
max_data=-1#10000
# define input sequence
seq_in = train_data['english_emb'].values[:max_data]#array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
seq_in = np.array([np.array(line) for line in seq_in])
# reshape input into [samples, timesteps, features]
seq_in = seq_in.reshape((len(seq_in),max_sent_eng,embedding_dim))
# prepare output sequence
seq_out = train_data['italian_emb'].values[:max_data]#seq_in[:, 1:,:]
seq_out = np.array([np.array(line) for line in seq_out])
#max words in italial sentences
n_out = max_sent_it
# define model
model = Sequential()
model.add(LSTM(30, activation='relu', input_shape=(max_sent_eng,embedding_dim))) #30
model.add(RepeatVector(n_out))
model.add(LSTM(30, activation='relu', return_sequences=True)) #activation fun
model.add(TimeDistributed(Dense(embedding_dim)))
model.compile(optimizer='adam', loss='mse')

# fit model
model.fit(seq_in, seq_out, epochs=300, verbose=0)   #Epoches 注意!!



NameError: name 'train_data' is not defined

### Testing Model

In [283]:

max_data_test=2000
#print(test_data)
seq_in_test = test_data['english_emb'].values[:max_data_test]
seq_in_test = np.array([np.array(line) for line in seq_in_test])
seq_out_test = test_data['italian_emb'].values[:max_data_test]
seq_out_test = np.array([np.array(line) for line in seq_out_test])

yhat = model.predict(seq_in_test, verbose=0)

def get_sentence(embeddings,language):
    if language=='english':
        return ' '.join([embedding_eng.most_similar(positive=[emb], topn=1)[0][0] for emb in embeddings]).replace('\n','').replace('\t','')
    elif language=='italian':
        return ' '.join([embedding_it.most_similar(positive=[emb], topn=1)[0][0] for emb in embeddings]).replace('\n','').replace('\t','')

index=np.random.randint(0,high=max_data_test)
print("English:\t",get_sentence(seq_in_test[index],'english'))
print("True Italian:\t",get_sentence(seq_out_test[index],'italian'))
print("Translation:\t",get_sentence(yhat[index],'italian'))

English:	  Where's the mistake 
True Italian:	  Dov'è l'errore  
Translation:	  È il un'amputazione 


<ipython-input-283-d3915a94948b>:12: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  return ' '.join([embedding_eng.most_similar(positive=[emb], topn=1)[0][0] for emb in embeddings]).replace('\n','').replace('\t','')
<ipython-input-283-d3915a94948b>:14: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  return ' '.join([embedding_it.most_similar(positive=[emb], topn=1)[0][0] for emb in embeddings]).replace('\n','').replace('\t','')


### Even when model is wrong, it is kind of right, i.e., the translation usually gives us qualitatively similar meanings

In [257]:

plot_model(model, to_file='model.png')


('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


## Another example: predicting data 2 timesteps out

In [122]:
# taken from: https://machinelearningmastery.com/encoder-decoder-attention-sequence-to-sequence-prediction-keras/

# generate a sequence of random integers
def generate_sequence(length, n_unique):
	return [randint(0, n_unique-1) for _ in range(length)]

# one hot encode sequence
def one_hot_encode(sequence, n_unique):
	encoding = list()
	for value in sequence:
		vector = [0 for _ in range(n_unique)]
		vector[value] = 1
		encoding.append(vector)
	return array(encoding)

# decode a one hot encoded string
def one_hot_decode(encoded_seq):
	return [argmax(vector) for vector in encoded_seq]

# prepare data for the LSTM
def get_pair(n_in, n_out, cardinality):
	# generate random sequence
	sequence_in = generate_sequence(n_in, cardinality)
	sequence_out = sequence_in[:n_out] + [0 for _ in range(n_in-n_out)]
	# one hot encode
	X = one_hot_encode(sequence_in, cardinality)
	y = one_hot_encode(sequence_out, cardinality)
	# reshape as 3D
	X = X.reshape((1, X.shape[0], X.shape[1]))
	y = y.reshape((1, y.shape[0], y.shape[1]))
	return X,y

# configure problem
n_features = 50
n_timesteps_in = 5
n_timesteps_out = 2
# define model
model = Sequential()
model.add(LSTM(50, input_shape=(n_timesteps_in, n_features)))
model.add(RepeatVector(n_timesteps_in))
model.add(LSTM(50, return_sequences=True))
model.add(TimeDistributed(Dense(n_features, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# train LSTM
for epoch in range(5000):
	# generate new random sequence
	X,y = get_pair(n_timesteps_in, n_timesteps_out, n_features)
	# fit model for one epoch on this sequence
	model.fit(X, y, epochs=1, verbose=0)
# evaluate LSTM
total, correct = 100, 0
for _ in range(total):
	X,y = get_pair(n_timesteps_in, n_timesteps_out, n_features)
	yhat = model.predict(X, verbose=0)
	if array_equal(one_hot_decode(y[0]), one_hot_decode(yhat[0])):
		correct += 1


1/1 - 2s - loss: 3.9101 - accuracy: 0.0000e+00
1/1 - 0s - loss: 3.8983 - accuracy: 0.0000e+00
1/1 - 0s - loss: 3.8978 - accuracy: 0.6000
1/1 - 0s - loss: 3.8894 - accuracy: 0.6000
1/1 - 0s - loss: 3.8792 - accuracy: 0.6000
1/1 - 0s - loss: 3.8620 - accuracy: 0.6000
1/1 - 0s - loss: 3.8490 - accuracy: 0.6000
1/1 - 0s - loss: 3.8633 - accuracy: 0.6000
1/1 - 0s - loss: 3.8082 - accuracy: 0.6000
1/1 - 0s - loss: 3.7880 - accuracy: 0.6000
1/1 - 0s - loss: 3.7968 - accuracy: 0.6000
1/1 - 0s - loss: 3.7411 - accuracy: 0.6000
1/1 - 0s - loss: 3.7070 - accuracy: 0.8000
1/1 - 0s - loss: 3.6478 - accuracy: 0.6000
1/1 - 0s - loss: 3.6368 - accuracy: 0.6000
1/1 - 0s - loss: 3.5406 - accuracy: 0.6000
1/1 - 0s - loss: 3.4234 - accuracy: 0.6000
1/1 - 0s - loss: 3.3560 - accuracy: 0.6000
1/1 - 0s - loss: 3.4194 - accuracy: 0.6000
1/1 - 0s - loss: 3.1214 - accuracy: 0.8000
1/1 - 0s - loss: 2.9806 - accuracy: 0.6000
1/1 - 0s - loss: 2.6219 - accuracy: 0.6000
1/1 - 0s - loss: 2.3721 - accuracy: 0.6000
1/1

1/1 - 0s - loss: 1.5347 - accuracy: 0.6000
1/1 - 0s - loss: 1.6174 - accuracy: 0.6000
1/1 - 0s - loss: 1.6922 - accuracy: 0.6000
1/1 - 0s - loss: 1.6164 - accuracy: 0.6000
1/1 - 0s - loss: 1.7751 - accuracy: 0.6000
1/1 - 0s - loss: 1.5939 - accuracy: 0.6000
1/1 - 0s - loss: 1.5818 - accuracy: 0.6000
1/1 - 0s - loss: 1.6089 - accuracy: 0.6000
1/1 - 0s - loss: 1.6505 - accuracy: 0.6000
1/1 - 0s - loss: 1.5689 - accuracy: 0.6000
1/1 - 0s - loss: 1.6375 - accuracy: 0.6000
1/1 - 0s - loss: 1.5590 - accuracy: 0.6000
1/1 - 0s - loss: 1.7307 - accuracy: 0.6000
1/1 - 0s - loss: 1.6078 - accuracy: 0.6000
1/1 - 0s - loss: 1.6223 - accuracy: 0.6000
1/1 - 0s - loss: 1.5790 - accuracy: 0.6000
1/1 - 0s - loss: 1.5735 - accuracy: 0.6000
1/1 - 0s - loss: 1.1013 - accuracy: 0.8000
1/1 - 0s - loss: 1.4708 - accuracy: 0.6000
1/1 - 0s - loss: 1.5091 - accuracy: 0.6000
1/1 - 0s - loss: 1.6740 - accuracy: 0.6000
1/1 - 0s - loss: 1.5932 - accuracy: 0.6000
1/1 - 0s - loss: 1.6304 - accuracy: 0.6000
1/1 - 0s - 

1/1 - 0s - loss: 1.5899 - accuracy: 0.6000
1/1 - 0s - loss: 1.5789 - accuracy: 0.6000
1/1 - 0s - loss: 1.5735 - accuracy: 0.6000
1/1 - 0s - loss: 1.5410 - accuracy: 0.6000
1/1 - 0s - loss: 1.6941 - accuracy: 0.6000
1/1 - 0s - loss: 1.5999 - accuracy: 0.6000
1/1 - 0s - loss: 1.6426 - accuracy: 0.6000
1/1 - 0s - loss: 1.6403 - accuracy: 0.8000
1/1 - 0s - loss: 1.5008 - accuracy: 0.6000
1/1 - 0s - loss: 1.5688 - accuracy: 0.6000
1/1 - 0s - loss: 1.6578 - accuracy: 0.6000
1/1 - 0s - loss: 1.5372 - accuracy: 0.6000
1/1 - 0s - loss: 1.5601 - accuracy: 0.6000
1/1 - 0s - loss: 1.5885 - accuracy: 0.6000
1/1 - 0s - loss: 1.5633 - accuracy: 0.6000
1/1 - 0s - loss: 1.5380 - accuracy: 0.6000
1/1 - 0s - loss: 1.5457 - accuracy: 0.6000
1/1 - 0s - loss: 1.1630 - accuracy: 0.8000
1/1 - 0s - loss: 1.5579 - accuracy: 0.6000
1/1 - 0s - loss: 1.7671 - accuracy: 0.6000
1/1 - 0s - loss: 1.6852 - accuracy: 0.6000
1/1 - 0s - loss: 1.5569 - accuracy: 0.6000
1/1 - 0s - loss: 1.5559 - accuracy: 0.6000
1/1 - 0s - 

1/1 - 0s - loss: 1.5683 - accuracy: 0.6000
1/1 - 0s - loss: 1.5785 - accuracy: 0.6000
1/1 - 0s - loss: 1.5398 - accuracy: 0.8000
1/1 - 0s - loss: 1.4826 - accuracy: 0.6000
1/1 - 0s - loss: 1.5022 - accuracy: 0.6000
1/1 - 0s - loss: 1.5500 - accuracy: 0.6000
1/1 - 0s - loss: 1.6456 - accuracy: 0.6000
1/1 - 0s - loss: 1.6139 - accuracy: 0.6000
1/1 - 0s - loss: 1.5806 - accuracy: 0.6000
1/1 - 0s - loss: 1.5883 - accuracy: 0.8000
1/1 - 0s - loss: 1.5725 - accuracy: 0.6000
1/1 - 0s - loss: 1.5884 - accuracy: 0.6000
1/1 - 0s - loss: 1.5807 - accuracy: 0.6000
1/1 - 0s - loss: 1.6409 - accuracy: 0.6000
1/1 - 0s - loss: 1.5914 - accuracy: 0.6000
1/1 - 0s - loss: 1.6478 - accuracy: 0.6000
1/1 - 0s - loss: 1.5805 - accuracy: 0.6000
1/1 - 0s - loss: 1.6876 - accuracy: 0.6000
1/1 - 0s - loss: 1.6343 - accuracy: 0.6000
1/1 - 0s - loss: 1.5811 - accuracy: 0.6000
1/1 - 0s - loss: 1.6124 - accuracy: 0.6000
1/1 - 0s - loss: 1.6606 - accuracy: 0.6000
1/1 - 0s - loss: 1.5634 - accuracy: 0.6000
1/1 - 0s - 

1/1 - 0s - loss: 1.5722 - accuracy: 0.6000
1/1 - 0s - loss: 1.6032 - accuracy: 0.6000
1/1 - 0s - loss: 1.4540 - accuracy: 0.6000
1/1 - 0s - loss: 1.4910 - accuracy: 0.6000
1/1 - 0s - loss: 1.5083 - accuracy: 0.8000
1/1 - 0s - loss: 1.4660 - accuracy: 0.6000
1/1 - 0s - loss: 1.4786 - accuracy: 0.6000
1/1 - 0s - loss: 1.4482 - accuracy: 0.6000
1/1 - 0s - loss: 1.6649 - accuracy: 0.6000
1/1 - 0s - loss: 1.5453 - accuracy: 0.6000
1/1 - 0s - loss: 1.5510 - accuracy: 0.6000
1/1 - 0s - loss: 1.5242 - accuracy: 0.6000
1/1 - 0s - loss: 1.5120 - accuracy: 0.6000
1/1 - 0s - loss: 1.5297 - accuracy: 0.6000
1/1 - 0s - loss: 1.4774 - accuracy: 0.6000
1/1 - 0s - loss: 1.5571 - accuracy: 0.6000
1/1 - 0s - loss: 1.5431 - accuracy: 0.8000
1/1 - 0s - loss: 1.7427 - accuracy: 0.6000
1/1 - 0s - loss: 1.5438 - accuracy: 0.6000
1/1 - 0s - loss: 1.6549 - accuracy: 0.6000
1/1 - 0s - loss: 1.5922 - accuracy: 0.6000
1/1 - 0s - loss: 1.5581 - accuracy: 0.6000
1/1 - 0s - loss: 1.5418 - accuracy: 0.6000
1/1 - 0s - 

...

1/1 - 0s - loss: 0.4345 - accuracy: 0.8000
1/1 - 0s - loss: 0.9717 - accuracy: 0.8000
1/1 - 0s - loss: 0.8766 - accuracy: 0.6000
1/1 - 0s - loss: 0.1142 - accuracy: 1.0000
1/1 - 0s - loss: 0.3025 - accuracy: 0.8000
1/1 - 0s - loss: 0.6292 - accuracy: 0.8000
1/1 - 0s - loss: 0.6462 - accuracy: 0.8000
1/1 - 0s - loss: 0.1517 - accuracy: 1.0000
1/1 - 0s - loss: 0.2435 - accuracy: 1.0000
1/1 - 0s - loss: 0.5110 - accuracy: 0.8000
1/1 - 0s - loss: 0.4704 - accuracy: 0.8000
1/1 - 0s - loss: 0.8911 - accuracy: 0.8000
1/1 - 0s - loss: 0.5964 - accuracy: 0.8000
1/1 - 0s - loss: 1.0801 - accuracy: 0.8000
1/1 - 0s - loss: 0.5374 - accuracy: 0.8000
1/1 - 0s - loss: 1.1320 - accuracy: 0.8000
1/1 - 0s - loss: 0.9537 - accuracy: 0.8000
1/1 - 0s - loss: 0.5529 - accuracy: 0.8000
1/1 - 0s - loss: 0.4709 - accuracy: 0.8000
1/1 - 0s - loss: 0.5853 - accuracy: 0.8000
1/1 - 0s - loss: 0.4409 - accuracy: 1.0000
1/1 - 0s - loss: 0.9106 - accuracy: 0.8000
1/1 - 0s - loss: 0.3158 - accuracy: 1.0000
1/1 - 0s - 

1/1 - 0s - loss: 0.7852 - accuracy: 0.8000
1/1 - 0s - loss: 0.5310 - accuracy: 0.8000
1/1 - 0s - loss: 0.6055 - accuracy: 0.8000
1/1 - 0s - loss: 0.3740 - accuracy: 1.0000
1/1 - 0s - loss: 0.7944 - accuracy: 0.8000
1/1 - 0s - loss: 0.4111 - accuracy: 0.8000
1/1 - 0s - loss: 0.5472 - accuracy: 0.8000
1/1 - 0s - loss: 1.0573 - accuracy: 0.8000
1/1 - 0s - loss: 0.4614 - accuracy: 0.8000
1/1 - 0s - loss: 0.5619 - accuracy: 0.8000
1/1 - 0s - loss: 1.3131 - accuracy: 0.6000
1/1 - 0s - loss: 0.5699 - accuracy: 0.6000
1/1 - 0s - loss: 0.6486 - accuracy: 0.8000
1/1 - 0s - loss: 0.2707 - accuracy: 1.0000
1/1 - 0s - loss: 0.3501 - accuracy: 1.0000
1/1 - 0s - loss: 0.5073 - accuracy: 0.8000
1/1 - 0s - loss: 0.5228 - accuracy: 1.0000
1/1 - 0s - loss: 0.7334 - accuracy: 0.8000
1/1 - 0s - loss: 0.3492 - accuracy: 1.0000
1/1 - 0s - loss: 0.8261 - accuracy: 0.8000
1/1 - 0s - loss: 0.6467 - accuracy: 0.8000
1/1 - 0s - loss: 0.5955 - accuracy: 0.8000
1/1 - 0s - loss: 0.8551 - accuracy: 0.8000
1/1 - 0s - 

1/1 - 0s - loss: 0.4504 - accuracy: 0.8000
1/1 - 0s - loss: 0.5312 - accuracy: 0.8000
1/1 - 0s - loss: 0.3153 - accuracy: 1.0000
1/1 - 0s - loss: 0.7893 - accuracy: 0.8000
1/1 - 0s - loss: 0.6098 - accuracy: 0.8000
1/1 - 0s - loss: 0.3842 - accuracy: 1.0000
1/1 - 0s - loss: 0.2635 - accuracy: 1.0000
1/1 - 0s - loss: 0.4719 - accuracy: 0.8000
1/1 - 0s - loss: 0.8049 - accuracy: 0.8000
1/1 - 0s - loss: 0.5832 - accuracy: 0.8000
1/1 - 0s - loss: 0.5806 - accuracy: 0.8000
1/1 - 0s - loss: 0.5139 - accuracy: 0.8000
1/1 - 0s - loss: 0.9191 - accuracy: 0.8000
1/1 - 0s - loss: 0.9907 - accuracy: 0.8000
1/1 - 0s - loss: 0.3251 - accuracy: 1.0000
1/1 - 0s - loss: 0.3790 - accuracy: 0.8000
1/1 - 0s - loss: 1.0514 - accuracy: 0.8000
1/1 - 0s - loss: 0.5872 - accuracy: 0.8000
1/1 - 0s - loss: 0.6603 - accuracy: 0.8000
1/1 - 0s - loss: 0.9311 - accuracy: 0.8000
1/1 - 0s - loss: 0.5272 - accuracy: 0.8000
1/1 - 0s - loss: 1.2030 - accuracy: 0.8000
1/1 - 0s - loss: 0.5768 - accuracy: 0.8000
1/1 - 0s - 

1/1 - 0s - loss: 0.3375 - accuracy: 1.0000
1/1 - 0s - loss: 0.2689 - accuracy: 1.0000
1/1 - 0s - loss: 0.6200 - accuracy: 0.8000
1/1 - 0s - loss: 0.8701 - accuracy: 0.8000
1/1 - 0s - loss: 0.7701 - accuracy: 0.8000
1/1 - 0s - loss: 0.8479 - accuracy: 0.8000
1/1 - 0s - loss: 0.1303 - accuracy: 1.0000
1/1 - 0s - loss: 0.0603 - accuracy: 1.0000
1/1 - 0s - loss: 0.0840 - accuracy: 1.0000
1/1 - 0s - loss: 0.8225 - accuracy: 0.8000
1/1 - 0s - loss: 0.4513 - accuracy: 0.8000
1/1 - 0s - loss: 0.4483 - accuracy: 0.8000
1/1 - 0s - loss: 0.5268 - accuracy: 0.8000
1/1 - 0s - loss: 0.2050 - accuracy: 1.0000
1/1 - 0s - loss: 0.2014 - accuracy: 1.0000
1/1 - 0s - loss: 0.5078 - accuracy: 0.8000
1/1 - 0s - loss: 0.3120 - accuracy: 0.8000
1/1 - 0s - loss: 0.5403 - accuracy: 0.8000
1/1 - 0s - loss: 0.6668 - accuracy: 0.8000
1/1 - 0s - loss: 0.5527 - accuracy: 0.8000
1/1 - 0s - loss: 0.4197 - accuracy: 0.8000
1/1 - 0s - loss: 0.4080 - accuracy: 0.8000
1/1 - 0s - loss: 0.8891 - accuracy: 0.8000
1/1 - 0s - 

1/1 - 0s - loss: 0.6847 - accuracy: 0.8000
1/1 - 0s - loss: 0.3129 - accuracy: 1.0000
1/1 - 0s - loss: 0.2297 - accuracy: 1.0000
1/1 - 0s - loss: 0.4135 - accuracy: 0.8000
1/1 - 0s - loss: 0.8284 - accuracy: 0.8000
1/1 - 0s - loss: 0.4352 - accuracy: 0.8000
1/1 - 0s - loss: 0.9910 - accuracy: 0.8000
1/1 - 0s - loss: 0.7248 - accuracy: 0.8000
1/1 - 0s - loss: 0.4503 - accuracy: 0.8000
1/1 - 0s - loss: 0.2998 - accuracy: 0.8000
1/1 - 0s - loss: 0.8147 - accuracy: 0.8000
1/1 - 0s - loss: 0.7088 - accuracy: 0.8000
1/1 - 0s - loss: 0.3074 - accuracy: 1.0000
1/1 - 0s - loss: 1.2133 - accuracy: 0.8000
1/1 - 0s - loss: 0.3235 - accuracy: 1.0000
1/1 - 0s - loss: 0.6528 - accuracy: 0.8000
1/1 - 0s - loss: 0.6130 - accuracy: 0.8000
1/1 - 0s - loss: 0.6912 - accuracy: 0.8000
1/1 - 0s - loss: 0.7690 - accuracy: 0.8000
1/1 - 0s - loss: 0.6803 - accuracy: 0.8000
1/1 - 0s - loss: 0.5772 - accuracy: 0.8000
1/1 - 0s - loss: 0.8926 - accuracy: 0.8000
1/1 - 0s - loss: 0.6908 - accuracy: 0.8000
1/1 - 0s - 

In [123]:
print('Accuracy: %.2f%%' % (float(correct)/float(total)*100.0))
# spot check some examples
for _ in range(10):
	X,y = get_pair(n_timesteps_in, n_timesteps_out, n_features)
	yhat = model.predict(X, verbose=0)
	print('Expected:', one_hot_decode(y[0]), 'Predicted', one_hot_decode(yhat[0]))


Accuracy: 19.00%
Expected: [41, 8, 0, 0, 0] Predicted [41, 41, 0, 0, 0]
Expected: [1, 45, 0, 0, 0] Predicted [1, 45, 0, 0, 0]
Expected: [40, 30, 0, 0, 0] Predicted [40, 12, 0, 0, 0]
Expected: [40, 21, 0, 0, 0] Predicted [40, 21, 0, 0, 0]
Expected: [20, 24, 0, 0, 0] Predicted [20, 20, 0, 0, 0]
Expected: [47, 19, 0, 0, 0] Predicted [47, 47, 0, 0, 0]
Expected: [19, 25, 0, 0, 0] Predicted [19, 19, 0, 0, 0]
Expected: [31, 18, 0, 0, 0] Predicted [31, 18, 0, 0, 0]
Expected: [5, 28, 0, 0, 0] Predicted [5, 29, 0, 0, 0]
Expected: [38, 39, 0, 0, 0] Predicted [38, 38, 0, 0, 0]
